## Section 1: Setup

In [1]:
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Gemini API key setup complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
    )

✅ Gemini API key setup complete.


### Import ADK components

In [2]:
from typing import Any, Dict

from google.adk.agents import Agent, LlmAgent
from google.adk.apps.app import App, EventsCompactionConfig
from google.adk.models.google_llm import Gemini
from google.adk.sessions import DatabaseSessionService
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.adk.tools.tool_context import ToolContext
from google.genai import types

print("ADK components imported successfully.")

ADK components imported successfully.


### Helper functions

In [3]:
# Define helper functions that will be reused throughout the notebook
async def run_session(
    runner_instance: Runner,
    user_queries: list[str] | str = None,
    session_name: str = "default",
):
    print(f"\n ### Session: {session_name}")

    # Get app name from the Runner
    app_name = runner_instance.app_name

    # Attempt to create a new session or retrieve an existing one
    try:
        session = await session_service.create_session(
            app_name=app_name, user_id=USER_ID, session_id=session_name
        )
    except:
        session = await session_service.get_session(
            app_name=app_name, user_id=USER_ID, session_id=session_name
        )

    # Process queries if provided
    if user_queries:
        # Convert single query to list for uniform processing
        if type(user_queries) == str:
            user_queries = [user_queries]

        # Process each query in the list sequentially
        for query in user_queries:
            print(f"\nUser > {query}")

            # Convert the query string to the ADK Content format
            query = types.Content(role="user", parts=[types.Part(text=query)])

            # Stream the agent's response asynchronously
            async for event in runner_instance.run_async(
                user_id=USER_ID, session_id=session.id, new_message=query
            ):
                # Check if the event contains valid content
                if event.content and event.content.parts:
                    # Filter out empty or "None" responses before printing
                    if (
                        event.content.parts[0].text != "None"
                        and event.content.parts[0].text
                    ):
                        print(f"{MODEL_NAME} > ", event.content.parts[0].text)
    else:
        print("No queries!")


print("Helper functions defined.")

Helper functions defined.


### Configure Retry Options

In [4]:
retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

---
## Section 2: Session Management
**Session**

A session is a container for conversations. It encapsulates the conversation history in a chronological manner and also records all tool interactions and responses for a single, continuous conversation. A session is tied to a user and agent; it is not shared with other users. Similarly, a session history for an Agent is not shared with other Agents.


### Implementing Stateful Agent

Let's build stateful agent, that can remember and have constructive conversations. 

In [5]:
APP_NAME = "default"  # Application
USER_ID = "default"  # User
SESSION = "default"  # Session

MODEL_NAME = "gemini-2.5-flash-lite"


# Step 1: Create the LLM Agent
root_agent = Agent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="text_chat_bot",
    description="A text chatbot",  # Description of the agent's purpose
)

# Step 2: Set up Session Management
# InMemorySessionService stores conversations in RAM (temporary)
session_service = InMemorySessionService()

# Step 3: Create the Runner
runner = Runner(agent=root_agent, app_name=APP_NAME, session_service=session_service)

print("Stateful agent initialized!")
print(f"   - Application: {APP_NAME}")
print(f"   - User: {USER_ID}")
print(f"   - Using: {session_service.__class__.__name__}")

Stateful agent initialized!
   - Application: default
   - User: default
   - Using: InMemorySessionService


### Testing the Stateful Agent

In [6]:
await run_session(
    runner,
    [
        "Hi, I am Yadnywalky! What is the capital of India?",
        "Hello! What is my name?",  
    ],
    "stateful-agentic-session",
)


 ### Session: stateful-agentic-session

User > Hi, I am Yadnywalky! What is the capital of India?
gemini-2.5-flash-lite >  Hi Yadnywalky! The capital of India is **New Delhi**.

User > Hello! What is my name?
gemini-2.5-flash-lite >  You are Yadnywalky!


**Success!** 🎉 The agent remembered your name because both queries were part of the same session. The Runner automatically maintained the conversation history.

---
## Section 3: Persistent Sessions with `DatabaseSessionService`

While `InMemorySessionService` is great for prototyping, real-world applications need conversations to survive restarts, crashes, and deployments. Let's level up to persistent storage!

### Implementing Persistent Sessions

In [7]:
# Step 1: Create the same agent
chatbot_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="text_chat_bot",
    description="A text chatbot with persistent memory",
)

# Step 2: Switch to DatabaseSessionService
# SQLite database will be created automatically
db_url = "sqlite:///my_agent_data.db"  # Local SQLite file
session_service = DatabaseSessionService(db_url=db_url)

# Step 3: Create a new runner with persistent storage
runner = Runner(agent=chatbot_agent, app_name=APP_NAME, session_service=session_service)

print("Upgraded to persistent sessions!")
print(f"   - Database: my_agent_data.db")
print(f"   - Sessions will survive restarts!")

Upgraded to persistent sessions!
   - Database: my_agent_data.db
   - Sessions will survive restarts!


### Test Run 1: Verifying Persistence

In this first test run, I'll start a new conversation with the session ID `test-db-session-01`.

In [9]:
await run_session(
    runner,
    ["Hi, I am Yadnywaly! What is the capital of the Japan?", "Hello! What is my name?"],
    "test-db-session-01",
)


 ### Session: test-db-session-01

User > Hi, I am Yadnywaly! What is the capital of the Japan?
gemini-2.5-flash-lite >  Hi Yadnywaly! The capital of Japan is Tokyo.

User > Hello! What is my name?
gemini-2.5-flash-lite >  Your name is Yadnywaly.


In [10]:
await run_session(
    runner,
    ["What is the capital of India?", "Hello! What is my name?"],
    "test-db-session-01",
)


 ### Session: test-db-session-01

User > What is the capital of India?
gemini-2.5-flash-lite >  The capital of India is New Delhi.

User > Hello! What is my name?
gemini-2.5-flash-lite >  Your name is Yadnywaly.


### Let's verify that the session data is isolated

As mentioned earlier, a session is private conversation between an Agent and a User . Let's run`run_session` with a different session name `test-db-session-02` to confirm this.


In [11]:
await run_session(
    runner, ["Hello! What is my name?"], "test-db-session-02"
)  


 ### Session: test-db-session-02

User > Hello! What is my name?
gemini-2.5-flash-lite >  I do not have access to your personal information, so I cannot tell you your name. 



### How are the events stored in the Database?

In [12]:
import sqlite3

def check_data_in_db():
    with sqlite3.connect("my_agent_data.db") as connection:
        cursor = connection.cursor()
        result = cursor.execute(
            "select app_name, session_id, author, content from events"
        )
        print([_[0] for _ in result.description])
        for each in result.fetchall():
            print(each)


check_data_in_db()

['app_name', 'session_id', 'author', 'content']
('default', 'test-db-session-01', 'user', '{"parts": [{"text": "Hi, I am Yadnywaly! What is the capital of the Japan?"}], "role": "user"}')
('default', 'test-db-session-01', 'text_chat_bot', '{"parts": [{"text": "Hi Yadnywaly! The capital of Japan is Tokyo."}], "role": "model"}')
('default', 'test-db-session-01', 'user', '{"parts": [{"text": "Hello! What is my name?"}], "role": "user"}')
('default', 'test-db-session-01', 'text_chat_bot', '{"parts": [{"text": "Your name is Yadnywaly."}], "role": "model"}')
('default', 'test-db-session-01', 'user', '{"parts": [{"text": "Hi, I am Yadnywaly! What is the capital of the Japan?"}], "role": "user"}')
('default', 'test-db-session-01', 'text_chat_bot', '{"parts": [{"text": "Hi Yadnywaly! The capital of Japan is Tokyo."}], "role": "model"}')
('default', 'test-db-session-01', 'user', '{"parts": [{"text": "Hello! What is my name?"}], "role": "user"}')
('default', 'test-db-session-01', 'text_chat_bot',

---
## Section 4: Context Compaction

As you can see, all the events are stored in full in the session Database. For a long, complex task, this list of events can become very large, leading to slower performance and higher costs.

For this I have used **Context Compaction** feature to see **how to automatically reduce the context that's stored in the Session.**

### Create an App for the agent

In [13]:
research_app_compacting = App(
    name="research_app_compacting",
    root_agent=chatbot_agent,
    # This is the new part!
    events_compaction_config=EventsCompactionConfig(
        compaction_interval=3,  # Trigger compaction every 3 invocations
        overlap_size=1,  # Keep 1 previous turn for context
    ),
)

db_url = "sqlite:///my_agent_data.db"  # Local SQLite file
session_service = DatabaseSessionService(db_url=db_url)

# Create a new runner for upgraded app
research_runner_compacting = Runner(
    app=research_app_compacting, session_service=session_service
)


print("Research App upgraded with Events Compaction!")

Research App upgraded with Events Compaction!


/tmp/ipykernel_48/3108707918.py:5: UserWarning: [EXPERIMENTAL] EventsCompactionConfig: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  events_compaction_config=EventsCompactionConfig(


### Running the Demo

In [14]:
# Turn 1
await run_session(
    research_runner_compacting,
    "What is the latest news about AI in healthcare?",
    "compaction_demo",
)

# Turn 2
await run_session(
    research_runner_compacting,
    "Are there any new developments in drug discovery?",
    "compaction_demo",
)

# Turn 3 - Compaction should trigger after this turn!
await run_session(
    research_runner_compacting,
    "Tell me more about the second development you found.",
    "compaction_demo",
)

# Turn 4
await run_session(
    research_runner_compacting,
    "Who are the main companies involved in that?",
    "compaction_demo",
)


 ### Session: compaction_demo

User > What is the latest news about AI in healthcare?
gemini-2.5-flash-lite >  Here's a summary of some of the latest and most significant news in AI in healthcare, covering various aspects:

**Key Areas of Advancement and News:**

*   **Drug Discovery and Development:**
    *   **Accelerated Discovery:** AI continues to be a game-changer in identifying potential drug candidates much faster than traditional methods. Companies are using AI to analyze vast biological datasets, predict molecular interactions, and design novel compounds.
    *   **Personalized Medicine:** AI is crucial for developing highly targeted therapies based on an individual's genetic makeup and disease profile. This includes identifying which patients are most likely to respond to specific treatments.
    *   **Clinical Trial Optimization:** AI is being used to improve patient recruitment for clinical trials, predict trial outcomes, and analyze trial data more efficiently, potential

### Verifying Compaction in the Session History

In [15]:
# Get the final session state
final_session = await session_service.get_session(
    app_name=research_runner_compacting.app_name,
    user_id=USER_ID,
    session_id="compaction_demo",
)

print("--- Searching for Compaction Summary Event ---")
found_summary = False
for event in final_session.events:
    # Compaction events have a 'compaction' attribute
    if event.actions and event.actions.compaction:
        print("\n SUCCESS! Found the Compaction Event:")
        print(f"  Author: {event.author}")
        print(f"\n Compacted information: {event}")
        found_summary = True
        break

if not found_summary:
    print(
        "\n No compaction event found. Try increasing the number of turns in the demo."
    )

--- Searching for Compaction Summary Event ---

 SUCCESS! Found the Compaction Event:
  Author: user

 Compacted information: model_version=None content=None grounding_metadata=None partial=None turn_complete=None finish_reason=None error_code=None error_message=None interrupted=None custom_metadata=None usage_metadata=None live_session_resumption_update=None input_transcription=None output_transcription=None avg_logprobs=None logprobs_result=None cache_metadata=None citation_metadata=None invocation_id='bee6e4cc-f965-428a-8cbd-7ba8d78470d1' author='user' actions=EventActions(skip_summarization=None, state_delta={}, artifact_delta={}, transfer_to_agent=None, escalate=None, requested_auth_configs={}, requested_tool_confirmations={}, compaction={'start_timestamp': 1763749497.003937, 'end_timestamp': 1763749507.083113, 'compacted_content': {'parts': [{'function_call': None, 'code_execution_result': None, 'executable_code': None, 'file_data': None, 'function_response': None, 'inline_data':

---
## Section 5: Working with Session State

In [16]:
# Define scope levels for state keys 
USER_NAME_SCOPE_LEVELS = ("temp", "user", "app")


# This demonstrates how tools can write to session state using tool_context.
# The 'user:' prefix indicates this is user-specific data.
def save_userinfo(
    tool_context: ToolContext, user_name: str, country: str
) -> Dict[str, Any]:
    """
    Tool to record and save user name and country in session state.

    Args:
        user_name: The username to store in session state
        country: The name of the user's country
    """
    # Write to session state using the 'user:' prefix for user data
    tool_context.state["user:name"] = user_name
    tool_context.state["user:country"] = country

    return {"status": "success"}


# This demonstrates how tools can read from session state.
def retrieve_userinfo(tool_context: ToolContext) -> Dict[str, Any]:
    """
    Tool to retrieve user name and country from session state.
    """
    # Read from session state
    user_name = tool_context.state.get("user:name", "Username not found")
    country = tool_context.state.get("user:country", "Country not found")

    return {"status": "success", "user_name": user_name, "country": country}


print("Tools created.")

Tools created.


### Creating an Agent with Session State Tools

In [17]:
# Configuration
APP_NAME = "default"
USER_ID = "default"
MODEL_NAME = "gemini-2.5-flash-lite"

# Create an agent with session state tools
root_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="text_chat_bot",
    description="""A text chatbot.
    Tools for managing user context:
    * To record username and country when provided use `save_userinfo` tool. 
    * To fetch username and country when required use `retrieve_userinfo` tool.
    """,
    tools=[save_userinfo, retrieve_userinfo],  # Provide the tools to the agent
)

# Set up session service and runner
session_service = InMemorySessionService()
runner = Runner(agent=root_agent, session_service=session_service, app_name="default")

print("Agent with session state tools initialized!")

Agent with session state tools initialized!


### Testing Session State 

In [18]:
# Test conversation demonstrating session state
await run_session(
    runner,
    [
        "Hi there, how are you doing today? What is my name?",  # Agent shouldn't know the name yet
        "My name is Yadnywalky. I'm from India.",  # Provide name - agent should save it
        "What is my name? Which country am I from?",  # Agent should recall from session state
    ],
    "state-demo-session",
)


 ### Session: state-demo-session

User > Hi there, how are you doing today? What is my name?
gemini-2.5-flash-lite >  Hello! I'm doing great. I'd love to tell you your name, but I don't have it stored. If you tell me your name and country, I can save it for you.

User > My name is Yadnywalky. I'm from India.


gemini-2.5-flash-lite >  It is nice to meet you, Yadnywalky! I have saved that you are from India.

User > What is my name? Which country am I from?


gemini-2.5-flash-lite >  Your name is Yadnywalky and you are from India.


### Inspecting Session State

In [19]:
# Retrieve the session and inspect its state
session = await session_service.get_session(
    app_name=APP_NAME, user_id=USER_ID, session_id="state-demo-session"
)

print("Session State Contents:")
print(session.state)
print("\n Notice the 'user:name' and 'user:country' keys storing our data!")

Session State Contents:
{'user:name': 'Yadnywalky', 'user:country': 'India'}

 Notice the 'user:name' and 'user:country' keys storing our data!


### Session State Isolation

In [20]:
# Start a completely new session - the agent won't know my name
await run_session(
    runner,
    ["Hi there, how are you doing today? What is my name?"],
    "new-isolated-session",
)



 ### Session: new-isolated-session

User > Hi there, how are you doing today? What is my name?
gemini-2.5-flash-lite >  Hello! I'm doing well, thank you for asking. I can't recall your name at the moment. If you tell me your name and country, I can save it for you.


### Cross-Session State Sharing

In [21]:
# Check the state of the new session
session = await session_service.get_session(
    app_name=APP_NAME, user_id=USER_ID, session_id="new-isolated-session"
)

print("New Session State:")
print(session.state)

New Session State:
{'user:name': 'Yadnywalky', 'user:country': 'India'}


---

## 🧹 Cleanup

In [22]:
# Clean up any existing database to start fresh 
import os

if os.path.exists("my_agent_data.db"):
    os.remove("my_agent_data.db")
print("Cleaned up old database files")

Cleaned up old database files




## ✅ Congratulations! 🎉